In [21]:
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

---

## 1. Price parsing

- `original_price`: Giá gốc (giá chưa giảm, đơn vị: VND)
- `sale_price`: Giá khuyến mãi (giá đã giảm, đơn vị: VND)
- `precent_discount`: Phần trăm số tiền đã giảm (đơn vị: %)

<pre>
- Có giá khuyến mãi: 
    &lt;fare-sale&gt; chứa giá khuyến mãi
        &lt;fareSmall&gt; chứa giá gốc và phần trăm ưu đãi
- Không có giá khuyến mãi:
    &lt;fare&gt; chứa giá gốc
        &lt;fareSmall&gt; không chứa dữ liệu
</pre>

In [22]:
def has_no_discount_price(block) -> bool:
    '''
    Phân loại chuyến xe này có giá khuyến mãi hay không \n
    True: không có giá khuyến mãi \n
    False: có giá khuyến mãi
    '''
    fare = block.find('div', class_='fare')      
    
    if fare: 
        return True  # khong co gia khuyen mai
    
    return False     # co gia khuyen mai

def parse_fare(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi không có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi = None, phần trăm khuyến mãi = None
    '''

    price_discounted = None
    percent_discount = None

    fare = block.find('div', class_='fare')
    price_original = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()    
    
    return {
        "price_original": price_original ,
        "price_discounted":price_discounted,
        "percent_discount": percent_discount
        }

def parse_fare_small(block):
    '''
    Dành cho trường hợp có khuyến mãi \n
    Trả về: giá gốc, phần trăm khuyến mãi
    '''

    fare_small = block.find('div', class_='fareSmall')

    price_original = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip() if fare_small else None
    try:
        if fare_small.find('div', class_='percent'):
            percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
        else:
            percent_discount = None
    except Exception:
        percent_discount = None
        
    return price_original, percent_discount

def parse_sale_price_info(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe khi có giá khuyến mãi \n
    Trả về: giá gốc, giá khuyến mãi, phần trăm khuyến mãi
    '''
    
    fare_sale = block.find('div', class_='fare-sale')
    price_original, percent_discount = parse_fare_small(block)
    price_discounted = None
    if fare_sale and fare_sale.get_text(strip=True):
        price_discounted = fare_sale.get_text(strip=True).strip()
        
    return {
        "price_original": price_original ,
        "price_discounted":price_discounted,
        "percent_discount": percent_discount
        }

In [23]:
def parse_price(block):
    '''
    Trích xuất dữ liệu về giá của chuyến xe \n
    Trả về: giá gốc, giá khuyến mãi (nếu có), phần trăm khuyến mãi (nếu có)
    '''
    
    if has_no_discount_price(block):
        return parse_fare(block)
    else:
        return parse_sale_price_info(block)

---

## 2. Bus info parsing

In [24]:
def parser_trip_bus_info(container):
    '''
    Trích xuất thông tin từ một container chứa thông tin chuyến đi. \n
    Trả về Tuple: tên nhà xe, đánh giá nhà xe, loại ghế.
    '''
    
    # bus name / company name
    bus_element = container.find('div', class_='bus-name')
    company_name = bus_element.get_text(strip=True) if bus_element else None

    # bus rating
    rating_element = container.find('div', class_='bus-rating').find('span')
    bus_rating = rating_element.get_text(strip=True) if rating_element else None

    # seat_type
    seat_type = container.find('div', class_='seat-type')
    seat_type = seat_type.get_text(strip=True) if seat_type else None

    return {
        'company_name': company_name,
        'bus_rating': bus_rating,
        'seat_type': seat_type
    }

---

## 3. Route parsing

In [25]:
# Dữ liệu này nằm ở ô filter chuyến đi

def parse_route_info(block):
    '''
    Trích xuất dữ liệu từ filter của trang web \n
    Trả về: ngày khởi hành, nơi xuất phát (thành phố hiện tại), nơi đến (nơi đặt vé đến)
    '''

    departure_date, start_point, destination = None, None, None

    try:
        departure_date = block.find('p', class_='date-input-value').get_text(strip=True)
        start_point = block.find(id="from_input").get('value')
        destination = block.find(id="to_input").get('value')
    except Exception:
        pass

    return {
        'departure_date': departure_date,
        'start_point': start_point,
        'destination': destination
    }

---

## 4. Details trip info

### 4.1 Departure

In [26]:
# Dữ liệu này nằm trong container > 'from_content'

def parse_departure_trip_info(from_content):
    """
    Trích xuất thông tin điểm đi từ một container 'from_content'.
    Trả về một tuple chứa: giờ khởi hành, địa điểm đón khách.
    """
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not from_content:
        return None, None

    # departure time
    departure_time_element = from_content.find('div', class_='hour')
    departure_time = departure_time_element.get_text(strip=True) if departure_time_element else None

    # departure place
    from_place_tag = from_content.find('div', class_='place')
    pickup_point = from_place_tag.get_text(strip=True) if from_place_tag else None
    
    return {
        'departure_time': departure_time,
        'pickup_point': pickup_point
    }

### 4.2 Arrival

In [27]:
def parse_arrival_trip_info(to_content):
    """
    Trích xuất thông tin điểm đến từ một container 'to_content'.\n
    Trả về một tuple chứa: ngày đến, thời gian đến, điểm trả khách.
    """
    
    # nếu container không tồn tại, trả về giá trị None cho tất cả
    if not to_content:
        return None, None, None

    # lấy ngày đến
    date_arrival_tag = to_content.find('span', class_="text-date-arrival-time")
    arrival_date = date_arrival_tag.get_text(strip=True) if date_arrival_tag else None
    
    
    # lấy giờ và địa điểm trả khách
    content_to_info = to_content.find('div', class_='content-to-info')
    if content_to_info:
        to_hour_tag = content_to_info.find('div', class_='hour')
        arrival_time = to_hour_tag.get_text(strip=True) if to_hour_tag else None
        
        dropoff_place_element = content_to_info.find('div', class_='place')
        dropoff_point = dropoff_place_element.get_text(strip=True) if dropoff_place_element else None
        
    return {
        'arrival_date': arrival_date,
        'arrival_time': arrival_time,
        'dropoff_point': dropoff_point
    }


---

In [28]:
def parse_trip_timing(container):
    """
    Trích xuất thông tin chi tiết về chuyến đi (giờ, nơi đi - đến, thời gian di chuyển). \n
    Trả về: dict chứa thông tin khởi hành, điểm đến và thời lượng chuyến.
    """
    
    # Tìm khối chứa thông tin đi và đến
    from_to_content = container.find('div', class_="from-to-content")

    # Nếu không tìm thấy, trả về dict rỗng có cấu trúc sẵn
    if not from_to_content:
        return {
            "duration": None,
            "departure_time": None,
            "pickup_point": None,
            "departure_date": None,
            "arrival_date": None,
            "arrival_time": None,
            "dropoff_point": None,
        }

    # Lấy thông tin nơi khởi hành
    from_content = from_to_content.find('div', class_='content from')
    dict_departure_info = parse_departure_trip_info(from_content)

    # Lấy thông tin nơi đến
    to_content = from_to_content.find('div', class_='content to')
    dict_arrival_info = parse_arrival_trip_info(to_content)

    # Lấy thời gian di chuyển
    duration_tag = from_to_content.find('div', class_="duration")
    duration = duration_tag.get_text(strip=True) if duration_tag else None

    # Gộp toàn bộ thông tin lại
    trip_data = dict_departure_info | dict_arrival_info | {'duration': duration}
    
    return trip_data


In [29]:
def compile_trip_info(block):
    '''
    Tập hợp toàn bộ thông tin của 1 chuyến xe từ 1 khối dữ liệu (block). \n
    Trả về: dict chứa thông tin xe, lịch trình và giá vé.
    '''
    
    # Lấy thông tin chính của nhà xe
    dict_bus_info = parser_trip_bus_info(block)
    
    # Lấy thông tin giờ đi - giờ đến, điểm đón - trả
    dict_trip_details = parse_trip_timing(block)
    
    # Lấy thông tin giá vé (giá gốc, giá khuyến mãi)
    dict_price = parse_price(block)
    
    # Gộp tất cả dữ liệu vào 1 dictionary duy nhất
    trip_data = dict_bus_info | dict_trip_details | dict_price
    
    return trip_data


---

## Parsing rating

In [30]:
def parse_ratings_from_container(container):
    '''
    Trích xuất thông tin đánh giá (rating) của từng nhà xe trong 1 container. \n
    Trả về: list các cặp (rate_title, rate_point) hoặc [(None, None)] nếu không có dữ liệu.
    '''
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')   # Tìm tất cả khối chứa thông tin đánh giá
        
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')   # Mỗi phần tử chứa tiêu đề và điểm
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)   # Tiêu đề đánh giá
                rate_point = rate_ps[1].get_text(strip=True)   # Điểm đánh giá
                ratings.append((rate_title, rate_point))       # Lưu vào danh sách
        
        if ratings:
            return ratings     # Trả về danh sách nếu có dữ liệu
        else:
            return [(None, None)]   # Không có dữ liệu đánh giá
            
    except Exception:
        return [(None, None)]   # Trường hợp lỗi vẫn trả về giá trị mặc định


total

In [31]:
def extract_all_trips(soup):
    '''
    Trích xuất và gộp thông tin chuyến xe, tuyến đường và đánh giá nhà xe thành một DataFrame duy nhất.
    Trả về: DataFrame chứa toàn bộ dữ liệu chuyến xe.
    '''
    dict_route = parse_route_info(soup)  # Lấy thông tin tuyến đường (đi - đến)
    containers = soup.find_all("div", class_="container")  # Tìm tất cả container chứa chuyến xe

    lst_trips_info = []

    for container in containers:
        # ---- Lấy thông tin chuyến xe ----
        dict_trip_info = compile_trip_info(container) | dict_route

        # ---- Lấy thông tin đánh giá ----
        ratings = parse_ratings_from_container(container)

        # ratings là list các tuple [(rate_title, rate_point), ...]
        # ta chuyển nó thành dict dạng {'rating_TênTiêuĐề': điểm}
        rating_dict = {
            f"{title}": point
            for title, point in ratings
            if title is not None and point is not None
        }

        # ---- Gộp thông tin chuyến xe và rating ----
        full_info = dict_trip_info | rating_dict

        # ---- Đưa vào DataFrame ----
        df_trip_info = pd.DataFrame([full_info])
        lst_trips_info.append(df_trip_info)

    # ---- Gộp toàn bộ chuyến xe lại thành 1 dataframe ----
    all_trips_info = pd.concat(lst_trips_info, ignore_index=True)

    return all_trips_info


---

In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException, StaleElementReferenceException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 5. Handle Button logic

### 5.1. Button `Xem thêm chuyến`

In [33]:
def click_load_more(driver, max_click: int = 8, max_wait: int = 15):
    """
    Click 'Xem thêm chuyến' tối đa max_click lần, dừng nếu hết nút.
    """
    wait = WebDriverWait(driver, max_wait)
    click_count = 0

    for _ in range(max_click):
        btns = driver.find_elements(By.CLASS_NAME, "load-more")
        if not btns:
            print("Không còn nút 'Xem thêm chuyến'")
            break

        try:
            btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "load-more")))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            btn.click()
            click_count += 1
            print(f"Đã click xem thêm lần {click_count}")

            # Chờ nút biến mất rồi xuất hiện lại
            wait.until(EC.staleness_of(btn))
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "load-more")))

        except TimeoutException:
            print("Không thấy nút sau khi chờ → dừng.")
            break


### 5.2. Button `Xem các đánh giá`

In [34]:
def expand_ratings(driver, click_prob=0.65, delay_range=(0.5, 1.2)):
    """1
    Mở ngẫu nhiên các phần đánh giá (rating) trên trang Vexere.
    - Không giới hạn số lần click.
    - Random bỏ qua 1 số nút để giả lập hành vi người thật.
    
    Parameters
    ----------
    driver : webdriver
        Trình duyệt Selenium đang điều khiển.
    click_prob : float, optional
        Xác suất click nút (0–1). Mặc định = 0.65 → click khoảng 65% số nút.
    delay_range : tuple, optional
        Khoảng nghỉ ngẫu nhiên giữa các lần click (giây).
    """
    try:
        wait = WebDriverWait(driver, 10)
        stars = wait.until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "bus-rating-button"))
        )

        total = len(stars)
        clicks = 0
        print(f"Tìm thấy {total} nút đánh giá.")

        for i, star in enumerate(stars, start=1):
            # Random quyết định có click hay không
            if random.random() > click_prob:
                print(f"Bỏ qua nút {i}/{total}")
                continue

            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
                time.sleep(random.uniform(*delay_range))
                driver.execute_script("arguments[0].click();", star)
                clicks += 1
                print(f"Click {i}/{total} (tổng {clicks} lần click)")
                time.sleep(random.uniform(*delay_range))
            except Exception as e:
                print(f"Lỗi khi click nút {i}/{total}: {e}")
                continue


    except Exception as e:
        print("Không thể mở phần đánh giá:", e)

### 5.3. Button `Tìm kiếm`

In [35]:
def click_search(driver, retries=3):
    """
    Click vào nút tìm kiếm trên trang Vexere (an toàn, tự scroll, retry nếu lỗi).
    """
    wait = WebDriverWait(driver, 10)
    
    for _ in range(retries):
        try:
            btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "button-search")))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", btn)
            print("Click vào nút tìm kiếm thành công")
            return True
        except Exception:
            print("Click tìm kiếm lỗi, thử lại")
            time.sleep(1)

    print("Không thể click vào nút tìm kiếm")
    return False

## 6. Automate the process of filtering website data

In [36]:
def get_target_date_components(days=0):
    """
    Trả về ngày và tháng-năm mục tiêu cách hiện tại `days` ngày.

    Parameters
    ----------
    days : int, optional
        Số ngày cộng thêm từ ngày hiện tại (mặc định = 0).

    Returns
    -------
    dict
        {'day': '15', 'month_year': '10-2025'}
    """
    # Ngày mục tiêu = ngày hiện tại + days ngày
    target_date = datetime.today() + timedelta(days=days)
    month_year = f"{target_date.month:02d}-{target_date.year}"
    day = str(target_date.day)

    return {
        'day': day,
        'month_year': month_year
    }

In [37]:
def set_search_filters(driver, start_city: str, destination_city: str, days=0):
    """
    Chọn điểm đi, điểm đến và ngày khởi hành trên trang Vexere.

    Parameters
    ----------
    driver : webdriver
        Đối tượng Selenium WebDriver đang điều khiển trình duyệt.
    start_city : str
        Tên thành phố khởi hành.
    destination_city : str
        Tên thành phố điểm đến.
    days : int, optional
        Số ngày tính từ hôm nay để chọn ngày đi (mặc định = 0).

    Returns
    -------
    bool
        True nếu chọn ngày thành công, False nếu xảy ra lỗi.
    """

    wait = WebDriverWait(driver, 10)

    try:
        # Nhập nơi đi và nơi đến
        departure_input = wait.until(EC.presence_of_element_located((By.ID, 'from_input')))
        destination_input = wait.until(EC.presence_of_element_located((By.ID, 'to_input')))

        departure_input.clear()
        destination_input.clear()

        departure_input.send_keys(start_city)
        destination_input.send_keys(destination_city)
        time.sleep(0.5)

        # Mở phần chọn ngày đi
        date_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "departure-date-select")))
        date_btn.click()
        time.sleep(1)

        # Lấy thông tin ngày và tháng cần chọn
        target = get_target_date_components(days)
        target_day = target['day']
        target_month = target['month_year']

        # Tìm khối tháng (có thể dùng '-' hoặc '_')
        try:
            month_section = driver.find_element(By.ID, target_month)
        except:
            month_section = driver.find_element(By.ID, target_month.replace('-', '_'))

        # Tìm tất cả các phần tử ngày trong tháng
        day_elements = month_section.find_elements(By.CSS_SELECTOR, "p.day")

        for day in day_elements:
            if day.text.strip() == target_day:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", day)
                time.sleep(0.3)
                driver.execute_script("arguments[0].click();", day)
                return True

        print("Không tìm thấy ngày cần chọn.")
        return False

    except Exception as e:
        print(f"Lỗi khi chọn bộ lọc tìm kiếm: {e}")
        return False

# FLOW OFFICIAL

---

## Crawl rating data

# -- Main --

In [38]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [39]:
URL = 'https://vexere.com/'

# Danh sách tuyến
arrivals_HaNoi = ['Hải Phòng','Nghệ An','Sơn La','Hà Giang','Quảng Ninh','Thanh Hóa','SaPa','Ninh Bình']
arrivals_SaiGon = ['Gia Lai','Bình Thuận','Ninh Thuận','Đắk Lắk','Phú Yên','Nha Trang - Khánh Hoà','Bà Rịa - Vũng Tàu']

departure_city = 'Sài Gòn'
days_offset = int(input(""))

# Lấy ngày / tháng cần crawl
target_date = get_target_date_components(days_offset)
day = target_date['day']
month_year = target_date['month_year']
month_years = month_year.replace('-', '_')

# Crawl từng tuyến
for arrival_city in arrivals_SaiGon:

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(URL)

    print(f"Đang crawl: {departure_city} → {arrival_city}")

    filter_success = set_search_filters(
        driver,
        start_city=departure_city,
        destination_city=arrival_city,
        days=days_offset
    )

    if not filter_success:
        print(f"Không thể chọn ngày cho {arrival_city}")
        continue

    time.sleep(2)
    click_search(driver)    # Click tìm kiếm sau khi lọc dữ liệu
    time.sleep(2.5)
    click_load_more(driver) # Click xem thêm để thấy đc nhiều chuyến xe
    
    expand_ratings(driver)  # Mở các thông tin đánh giá từng chuyến
    time.sleep(1.5)

    # Parse dữ liệu HTML
    soup = BeautifulSoup(driver.page_source, "html.parser")

    try:
        df_trips_info = extract_all_trips(soup)

        save_path = f"../data/raw/{departure_city}_{arrival_city}_{day}_{month_years}.csv"
        df_trips_info.to_csv(save_path, index=False, encoding='utf-8')

        print(f"Lấy dữ liệu {arrival_city} thành công! → {save_path}")

    except Exception as e:
        html_path = f"../data/site/{departure_city}_{arrival_city}_{day}_{month_years}.html"
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(soup.prettify())
        print(f"Lưu HTML để debug: {html_path}\n{e}")

# Đóng trình duyệt sau khi hoàn tất
driver.quit()

Đang crawl: Sài Gòn → Gia Lai
Click vào nút tìm kiếm thành công


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (660, 7249)
  (Session info: chrome=141.0.7390.123); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
	GetHandleVerifier [0x0xd2fe43+66515]
	GetHandleVerifier [0x0xd2fe84+66580]
	(No symbol) [0x0xb1dc48]
	(No symbol) [0x0xb6ec10]
	(No symbol) [0x0xb6cf73]
	(No symbol) [0x0xb6aa17]
	(No symbol) [0x0xb69c8d]
	(No symbol) [0x0xb5e115]
	(No symbol) [0x0xb8b1cc]
	(No symbol) [0x0xb5db74]
	(No symbol) [0x0xb8b384]
	(No symbol) [0x0xbacba7]
	(No symbol) [0x0xb8afc6]
	(No symbol) [0x0xb5c2ca]
	(No symbol) [0x0xb5d154]
	GetHandleVerifier [0x0xf87353+2521315]
	GetHandleVerifier [0x0xf822d3+2500707]
	GetHandleVerifier [0x0xd57c94+229924]
	GetHandleVerifier [0x0xd481f8+165768]
	GetHandleVerifier [0x0xd4ecad+193085]
	GetHandleVerifier [0x0xd38158+100072]
	GetHandleVerifier [0x0xd382f